In [ ]:
from database_handler import *

In [ ]:
session = create_connection()

In [ ]:
from transformation_handler import *

In [ ]:
mydata = clean_all_data(1000000)

In [ ]:
import datetime
sonoma = f"https://data.sonomacounty.ca.gov/resource/924a-vesw.json?$select=id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type"

In [ ]:
print(datetime.datetime.now())

In [ ]:
response = requests.get(sonoma, params={'$limit': 1000000,'$where': "intake_date >= '2023-10-08' or outcome_date >= '2023-10-08'",'$order': 'intake_date ASC'})
df = pd.DataFrame(response.json())

In [2]:
from transformation_handler import *

In [1]:
from prehook import *
from hook import *
from lookups import *
from database_handler import *
import requests

In [2]:
execute_prehook()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.
Closing connection to database...
Connection is closed.


In [3]:
execute_hook()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.


c:\Users\olasm\Desktop\FSD - July 2023\week 9\FSD Final Project\database_handler.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_path)


Error doing pandas transformations on Sonoma shelter dataset = "None of [Index(['intake_date', 'date_of_birth', 'outcome_date'], dtype='object')] are in the [columns]".
Error doing pandas transformations on Austin shelter dataset = 'intake_date'.
Error doing pandas transformations on Norfolk shelter dataset = 'years_old'.
Error doing pandas transformations on Bloomington shelter dataset = 'intake_date'.
Error doing pandas transformations on Dallas shelter datasets = 'intake_date'.
Error in: clean_all_data = "None of [Index(['type', 'breed', 'color', 'intake_type', 'sex', 'outcome_type'], dtype='object')] are in the [columns]".
Closing connection to database...
Connection is closed.


In [4]:
db_session = create_connection()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.


In [15]:
etl_date, exists = return_etl_last_updated_date(db_session)
etl_date

c:\Users\olasm\Desktop\FSD - July 2023\week 9\FSD Final Project\database_handler.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_path)


Timestamp('1900-10-08 00:00:00')

In [12]:
insert_data_into_stg_tables(db_session,DataWareHouseSchema.SCHEMA_NAME, etl_date.strftime("%Y-%m-%dT%H:%M:%S.%f"))

Error doing pandas transformations on Sonoma shelter dataset = unhashable type: 'dict'.
Error doing pandas transformations on Austin shelter dataset = 'intake_date'.
Error doing pandas transformations on Norfolk shelter dataset = 'years_old'.
Error doing pandas transformations on Bloomington shelter dataset = 'intake_date'.
Error doing pandas transformations on Dallas shelter datasets = unhashable type: 'dict'.
Error in: clean_all_data = "None of [Index(['type', 'breed', 'color', 'intake_type', 'sex', 'outcome_type'], dtype='object')] are in the [columns]".


In [24]:
condition = DateCondition.OTHER.value.replace('0',f"'{etl_date}'")
response = requests.get(DataSources.SHELTER_SONOMA.value, params={'$limit': 1000000, '$where':condition})
result = (DataSources.SHELTER_SONOMA.name.lower(), pd.DataFrame(response.json()))
result[1]

,id,type,breed,color,sex,intake_date,outcome_date,intake_type,outcome_type,date_of_birth
0,A420084,DOG,GERM SHEPHERD,BROWN/BLACK,Spayed,2023-08-31T00:00:00.000,2023-10-02T00:00:00.000,STRAY,EUTHANIZE,NaN
1,A420188,CAT,DOMESTIC SH,BRN TABBY,Male,2023-09-05T00:00:00.000,2023-10-02T00:00:00.000,STRAY,DIED,2023-08-05T00:00:00.000
2,A420842,DOG,PIT BULL,BLACK/WHITE,Male,2023-10-02T00:00:00.000,NaN,QUARANTINE,NaN,2012-10-02T00:00:00.000
3,A420859,DOG,BASENJI/MIX,TAN/WHITE,Female,2023-10-03T00:00:00.000,NaN,STRAY,NaN,NaN
4,A420866,CAT,DOMESTIC SH,AGOUTI,Unknown,2023-10-03T00:00:00.000,2023-10-03T00:00:00.000,STRAY,TRANSFER,NaN
...,...,...,...,...,...,...,...,...,...,...
175,A421230,CAT,DOMESTIC SH,CALICO/BRN TABBY,Female,2023-10-14T00:00:00.000,NaN,STRAY,NaN,2023-08-14T00:00:00.000
176,A421238,DOG,CHIHUAHUA LH/MIX,WHITE,Female,2023-10-14T00:00:00.000,NaN,STRAY,NaN,2023-01-14T00:00:00.000
177,A421236,DOG,CHIHUAHUA SH/MIX,TAN/WHITE,Male,2023-10-14T00:00:00.000,NaN,STRAY,NaN,2021-10-14T00:00:00.000
178,A421237,DOG,CHIHUAHUA SH,TAN/WHITE,Female,2023-10-14T00:00:00.000,NaN,STRAY,NaN,2023-01-14T00:00:00.000


In [17]:
# source_dfs_dict = readData(etl_date = etl_date.strftime("%Y-%m-%dT%H:%M:%S.%f"), limit=10000000)
stg_dfs_dict = clean_all_data(source_dfs_dict)

In [16]:
for key, value in source_dfs_dict.items():
    print(key)
    print(value)

shelter_sonoma
            id type                         breed            color       sex  \
0      A328255  DOG            MALTESE/POODLE TOY            WHITE    Spayed   
1      A420799  CAT                   DOMESTIC SH            BLACK   Unknown   
2      A420773  CAT                   DOMESTIC LH      BLACK/WHITE  Neutered   
3      A420810  CAT                   DOMESTIC SH        BRN TABBY      Male   
4      A417889  CAT                   DOMESTIC SH  ORG TABBY/WHITE    Spayed   
...        ...  ...                           ...              ...       ...   
26375  A320109  DOG                  PIT BULL/MIX  CHOCOLATE/WHITE    Spayed   
26376  A310168  CAT                   DOMESTIC SH  BRN TABBY/WHITE  Neutered   
26377  A345933  DOG             GERM SHEPHERD/MIX            BROWN  Neutered   
26378  A412870  CAT                   DOMESTIC SH       BLUE CREAM    Spayed   
26379  A314617  DOG  LABRADOR RETR/PORT WATER DOG            BLACK    Spayed   

                 date_of

In [18]:
for key, value in stg_dfs_dict.items():
    print(key)
    print(value)

intakes_outcomes_sonoma
        animal_id type                         breed              color  \
0      SO-A328255  Dog            Maltese/Poodle Toy              White   
1      SO-A420799  Cat                   Domestic Sh              Black   
2      SO-A420773  Cat                   Domestic Lh        Black/White   
3      SO-A420810  Cat                   Domestic Sh        Brown Tabby   
4      SO-A417889  Cat                   Domestic Sh    Org Tabby/White   
...           ...  ...                           ...                ...   
26375  SO-A320109  Dog                  Pit Bull/Mix    Chocolate/White   
26376  SO-A310168  Cat                   Domestic Sh  Brown Tabby/White   
26377  SO-A345933  Dog             Germ Shepherd/Mix              Brown   
26378  SO-A412870  Cat                   Domestic Sh         Blue Cream   
26379  SO-A314617  Dog  Labrador Retr/Port Water Dog              Black   

            sex date_of_birth intake_date outcome_date intake_type  \
0    

In [13]:
from database_handler import *
from lookups import *
db_session = create_connection()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.


In [58]:
source_dfs_dict = readData(etl_date = '2023-10-05', limit=10000000)
stg_dfs_dict = clean_all_data(source_dfs_dict)

In [ ]:
for df_name,stg_df in stg_dfs_dict.items():
    if len(stg_df) > 0:
        insert_stmt = insert_into_sql_statement_from_df(stg_df, DataWareHouseSchema.SCHEMA_NAME.value, 'stg_'+df_name)
        execute_return = execute_query(db_session=db_session, query= insert_stmt)
        if execute_return != Errors.NO_ERROR:
            raise Exception(f"{HookSteps.INSERT_INTO_STG_TABLE.value}: error executing insert_stmt.")

In [59]:
source_dfs_dict.keys()

dict_keys(['shelter_sonoma', 'shelter_austin_intakes', 'shelter_austin_outcomes', 'shelter_norfolk', 'shelter_bloomington', 'shelter_dallas_2017_2018', 'shelter_dallas_2018_2019', 'shelter_dallas_2019_2020', 'shelter_dallas_2020_2021', 'shelter_dallas_2021_2022', 'shelter_dallas_2022_2023', 'per_capita_sonoma_income', 'per_capita_austin_income', 'per_capita_norfolk_income', 'per_capita_bloomington_monroe_income', 'per_capita_dallas_income', 'population_sonoma', 'population_austin', 'population_norfolk', 'population_bloomington_monroe', 'population_dallas', 'unemployment_rate_sonoma', 'unemployment_rate_austin', 'unemployment_rate_norfolk_city', 'unemployment_rate_bloomington', 'unemployment_rate_dallas'])

In [60]:
stg_dfs_dict.keys()

dict_keys(['intakes_outcomes_sonoma', 'intakes_outcomes_austin', 'intakes_outcomes_norfolk', 'intakes_outcomes_bloomington', 'intakes_outcomes_dallas', 'per_capita_sonoma_income', 'per_capita_austin_income', 'per_capita_norfolk_income', 'per_capita_bloomington_monroe_income', 'per_capita_dallas_income', 'population_sonoma', 'population_austin', 'population_norfolk', 'population_bloomington_monroe', 'population_dallas', 'unemployment_rate_sonoma', 'unemployment_rate_austin', 'unemployment_rate_norfolk_city', 'unemployment_rate_bloomington', 'unemployment_rate_dallas'])

In [110]:
sonoma = source_dfs_dict['shelter_sonoma'].copy()
sonoma_cleaned = stg_dfs_dict['intakes_outcomes_sonoma'].copy()

In [114]:
sonoma_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   animal_id      118 non-null    object        
 1   type           118 non-null    object        
 2   breed          118 non-null    object        
 3   color          118 non-null    object        
 4   sex            118 non-null    object        
 5   date_of_birth  118 non-null    datetime64[ns]
 6   intake_date    118 non-null    datetime64[ns]
 7   outcome_date   118 non-null    datetime64[ns]
 8   intake_type    118 non-null    object        
 9   outcome_type   118 non-null    object        
 10  region         118 non-null    object        
dtypes: datetime64[ns](3), object(8)
memory usage: 11.1+ KB


In [112]:
columns = ['date_of_birth','intake_date','outcome_date']
sonoma_cleaned[columns] = sonoma_cleaned[columns].fillna('2262-04-11').astype('datetime64[ns]')
# sonoma_cleaned['date_of_birth'] = sonoma_cleaned['date_of_birth'].fillna('2262-04-11').astype('datetime64[ns]')
# sonoma_cleaned['intake_date'] = sonoma_cleaned['intake_date'].fillna('2262-04-11').astype('datetime64[ns]')
# sonoma_cleaned['outcome_date'] = sonoma_cleaned['outcome_date'].fillna('2262-04-11').astype('datetime64[ns]')

In [113]:
sonoma_cleaned

,animal_id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type,region
0,SO-A420761,Dog,French Bulldog,Tan,Female,2019-03-29,2023-10-06,2023-10-06,Stray,Return To Owner,Sonoma
1,SO-A420925,Dog,Doberman Pinsch,Black/Tan,Male,2022-04-06,2023-10-05,2023-10-06,Stray,Return To Owner,Sonoma
2,SO-A418625,Cat,Domestic Sh,Black/White,Neutered,2023-06-26,2023-06-27,2023-10-06,Stray,Adoption,Sonoma
3,SO-A420157,Cat,Domestic Mh,Tortie,Spayed,2023-03-25,2023-09-05,2023-10-06,Owner Surrender,Adoption,Sonoma
4,SO-A420031,Dog,Belg Malinois/Mix,Black/Tan,Neutered,2023-06-02,2023-08-29,2023-10-06,Stray,Adoption,Sonoma
...,...,...,...,...,...,...,...,...,...,...,...
113,SO-A421058,Cat,Domestic Sh,White/Black,Female,2023-09-10,2023-10-13,2023-10-13,Stray,Euthanize,Sonoma
114,SO-A421059,Cat,Domestic Sh,Brown Tabby/Black,Male,2023-09-10,2023-10-10,2262-04-11,Stray,Pending,Sonoma
115,SO-A419843,Dog,Welsh Corgi Pem/Cavalier Span,Black/White,Neutered,2021-08-22,2023-08-22,2023-10-13,Stray,Adoption,Sonoma
116,SO-A421205,Other,Chicken,Red,Unknown,2262-04-11,2023-10-13,2262-04-11,Confiscate,Pending,Sonoma


In [26]:
insert_stmt = insert_into_sql_statement_from_df(sonoma, DataWareHouseSchema.SCHEMA_NAME.value, 'stg_'+df_name)
# execute_return = execute_query(db_session=db_session, query= insert_stmt)

In [27]:
insert_stmt

"INSERT INTO dw_reporting_schema.stg_intakes_outcomes_sonoma (animal_id, type, breed, color, sex, date_of_birth, intake_date, outcome_date, intake_type, outcome_type, region) VALUES\n('SO-A420761', 'Dog', 'French Bulldog', 'Tan', 'Female', '2019-03-29 00:00:00', '2023-10-06 00:00:00', '2023-10-06 00:00:00', 'Stray', 'Return To Owner', 'Sonoma'),\n('SO-A420925', 'Dog', 'Doberman Pinsch', 'Black/Tan', 'Male', '2022-04-06 00:00:00', '2023-10-05 00:00:00', '2023-10-06 00:00:00', 'Stray', 'Return To Owner', 'Sonoma'),\n('SO-A418625', 'Cat', 'Domestic Sh', 'Black/White', 'Neutered', '2023-06-26 00:00:00', '2023-06-27 00:00:00', '2023-10-06 00:00:00', 'Stray', 'Adoption', 'Sonoma'),\n('SO-A420157', 'Cat', 'Domestic Mh', 'Tortie', 'Spayed', '2023-03-25 00:00:00', '2023-09-05 00:00:00', '2023-10-06 00:00:00', 'Owner Surrender', 'Adoption', 'Sonoma'),\n('SO-A420031', 'Dog', 'Belg Malinois/Mix', 'Black/Tan', 'Neutered', '2023-06-02 00:00:00', '2023-08-29 00:00:00', '2023-10-06 00:00:00', 'Stray',